# 糖尿病患者における心血管イベント発症リスクの検討

## 研究概要

### 背景
2型糖尿病患者は心血管疾患（心筋梗塞、脳卒中など）の発症リスクが高いことが知られている。特に日本では高齢化に伴い糖尿病患者数が増加しており、心血管イベントの予防は医療費抑制・QOL維持の観点から重要な課題。
先行研究では高血圧や慢性腎臓病（CKD）の併存が心血管リスクを高めることが示唆されているが、日本の大規模データベースを用いた検討は限定的。

### 研究目的
NDBの糖尿病患者コホートを用いて、以下を明らかにする：

- 糖尿病患者における併存疾患（高血圧、脂質異常症、慢性腎臓病）の保有状況
- 併存疾患パターン別の心血管イベント発症率の比較
- 心血管イベント発症に寄与するリスク因子の同定


### 解析手法 
- カプランマイヤー法による生存曲線の推定

In [0]:
%sql
-- データの基本情報を確認
SELECT 
  COUNT(*) AS total_patients,
  MIN(diabetes_diagnosis_date) AS earliest_diagnosis,
  MAX(diabetes_diagnosis_date) AS latest_diagnosis,
  COUNT(DISTINCT prefecture_code) AS prefecture_count
FROM ndb_poc.default.diabetes_cohort

In [0]:
# Sparkデータフレームとして読み込み
df = spark.table("ndb_poc.default.diabetes_cohort")

# スキーマ確認
print("=== データスキーマ ===")
df.printSchema()

# レコード数
print(f"\n総レコード数: {df.count():,}")

In [0]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Pandasデータフレームに変換
pdf = df.toPandas()

print(f"データ件数: {len(pdf):,}")
print(f"カラム数: {len(pdf.columns)}")
print(f"\n=== データ型 ===")
print(pdf.dtypes)

In [0]:
# 欠損値の件数と割合
missing_stats = pd.DataFrame({
    'missing_count': pdf.isnull().sum(),
    'missing_rate_pct': (pdf.isnull().sum() / len(pdf) * 100).round(2)
})
missing_stats = missing_stats[missing_stats['missing_count'] > 0].sort_values('missing_rate_pct', ascending=False)

print("=== 欠損値のあるカラム ===")
if len(missing_stats) > 0:
    display(missing_stats)
else:
    print("欠損値なし")

# 判定
max_missing = missing_stats['missing_rate_pct'].max() if len(missing_stats) > 0 else 0
print(f"\n最大欠損率: {max_missing}%")
print(f"判定: {'補完で対応可能' if max_missing < 10 else '要検討'}")

## 分析コホートの定義

### 選択基準
1. 診断時年齢40歳以上
2. 診断日が2018年12月31日以前（最低2年の観察期間確保）
3. 都道府県情報あり（地域調整のため）

### 除外基準
- 診断前に心血管イベント既往あり（本データでは該当なし）

In [0]:
# 分析コホートの作成
print("=== コホート選択の過程 ===")
print(f"全患者: {len(pdf):,}")

cohort = pdf.copy()

# 1. 40歳以上
cohort = cohort[cohort['age'] >= 40]
print(f"→ 40歳以上に限定: {len(cohort):,}")

# 2. 2018年以前の診断
cohort = cohort[pd.to_datetime(cohort['diabetes_diagnosis_date']) <= '2018-12-31']
print(f"→ 2018年以前の診断に限定: {len(cohort):,}")

# 3. 都道府県情報あり
cohort = cohort[cohort['prefecture_code'].notna()]
print(f"→ 都道府県情報ありに限定: {len(cohort):,}")

print(f"\n最終解析対象: {len(cohort):,}人")

In [0]:
#HbA1cとBMIの欠損値は中央値で補完


# HbA1c
hba1c_median = cohort['hba1c'].median()
hba1c_missing = cohort['hba1c'].isnull().sum()
cohort['hba1c_imputed'] = cohort['hba1c'].fillna(hba1c_median)
print(f"HbA1c: {hba1c_missing}件を中央値({hba1c_median})で補完")

# BMI
bmi_median = cohort['bmi'].median()
bmi_missing = cohort['bmi'].isnull().sum()
cohort['bmi_imputed'] = cohort['bmi'].fillna(bmi_median)
print(f"BMI: {bmi_missing}件を中央値({bmi_median})で補完")

# 補完後の確認
print(f"\n補完後の欠損数:")
print(f"  hba1c_imputed: {cohort['hba1c_imputed'].isnull().sum()}")
print(f"  bmi_imputed: {cohort['bmi_imputed'].isnull().sum()}")

In [0]:
# 派生変数の作成

# イベントフラグをint型に変換（生存分析用）
cohort['event'] = cohort['cv_event_occurred'].astype(int)

# Boolean型をint型に変換（集計用）
cohort['has_hypertension_int'] = cohort['has_hypertension'].astype(int)
cohort['has_dyslipidemia_int'] = cohort['has_dyslipidemia'].astype(int)
cohort['has_ckd_int'] = cohort['has_ckd'].astype(int)

# BMIカテゴリ
cohort['bmi_category'] = pd.cut(
    cohort['bmi_imputed'],
    bins=[0, 18.5, 25, 30, 100],
    labels=['Underweight', 'Normal', 'Overweight', 'Obese']
)

print("=== 派生変数の確認 ===")
print(f"イベント発生数: {cohort['event'].sum():,} ({cohort['event'].mean()*100:.2f}%)")
print(f"\nBMIカテゴリ分布:")
display(cohort['bmi_category'].value_counts())

In [0]:
# 分析コホートの概要
print("=" * 60)
print("分析コホートの概要")
print("=" * 60)

print(f"""
対象者数: {len(cohort):,}人
観察期間: {cohort['follow_up_months'].min()} - {cohort['follow_up_months'].max()}ヶ月
         （中央値: {cohort['follow_up_months'].median():.0f}ヶ月）

イベント発生:
  心血管イベント: {cohort['event'].sum():,}人 ({cohort['event'].mean()*100:.2f}%)

併存疾患:
  高血圧: {cohort['has_hypertension'].sum():,}人 ({cohort['has_hypertension'].mean()*100:.1f}%)
  脂質異常症: {cohort['has_dyslipidemia'].sum():,}人 ({cohort['has_dyslipidemia'].mean()*100:.1f}%)
  慢性腎臓病: {cohort['has_ckd'].sum():,}人 ({cohort['has_ckd'].mean()*100:.1f}%)
""")

In [0]:
# 併存疾患パターン別の集計
print("=== 併存疾患パターン別の患者数とイベント発生率 ===\n")

comorbidity_summary = cohort.groupby('comorbidity_pattern').agg({
    'patient_id': 'count',
    'event': ['sum', 'mean']
}).round(4)

comorbidity_summary.columns = ['n', 'events', 'event_rate']
comorbidity_summary['event_rate_pct'] = (comorbidity_summary['event_rate'] * 100).round(2)
comorbidity_summary['events'] = comorbidity_summary['events'].astype(int)
comorbidity_summary = comorbidity_summary.sort_values('n', ascending=False)

display(comorbidity_summary[['n', 'events', 'event_rate_pct']])

In [0]:
# lifelinesがインストールされていない場合はインストール
try:
    from lifelines import KaplanMeierFitter
    from lifelines.statistics import logrank_test
    print("lifelines is ready")
except ImportError:
    %pip install lifelines matplotlib_fontja
    from lifelines import KaplanMeierFitter
    from lifelines.statistics import logrank_test
    print("lifelines installed successfully")

In [0]:
import matplotlib.pyplot as plt
import matplotlib_fontja


plt.rcParams['font.size'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.figsize'] = (10, 6)

In [0]:
# カプランマイヤー分析: 高血圧有無別
kmf = KaplanMeierFitter()

fig, ax = plt.subplots(figsize=(10, 6))

# 高血圧なし群
mask_no_ht = ~cohort['has_hypertension']
kmf.fit(
    cohort.loc[mask_no_ht, 'follow_up_months'],
    cohort.loc[mask_no_ht, 'event'],
    label=f"Hypertension (-) (n={mask_no_ht.sum():,})"
)
kmf.plot_survival_function(ax=ax, ci_show=True, color='blue')

# 高血圧あり群
mask_ht = cohort['has_hypertension']
kmf.fit(
    cohort.loc[mask_ht, 'follow_up_months'],
    cohort.loc[mask_ht, 'event'],
    label=f"Hypertension (+) (n={mask_ht.sum():,})"
)
kmf.plot_survival_function(ax=ax, ci_show=True, color='red')

# グラフ装飾
ax.set_xlabel('Follow-up (months)')
ax.set_ylabel('Event-free Survival Rate')
ax.set_title('Cardiovascular Event-free Survival by Hypertension Status\n(Kaplan-Meier Curve)')
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)
ax.set_ylim(0.90, 1.0)  # イベント率が低いため拡大

plt.tight_layout()
display(fig)
plt.close()

In [0]:
# ログランク検定
results = logrank_test(
    cohort.loc[mask_no_ht, 'follow_up_months'],
    cohort.loc[mask_ht, 'follow_up_months'],
    cohort.loc[mask_no_ht, 'event'],
    cohort.loc[mask_ht, 'event']
)

print("=== ログランク検定結果 ===")
print(f"検定統計量 (χ²): {results.test_statistic:.4f}")
print(f"p値: {results.p_value:.2e}")
print(f"有意水準: α = 0.05")
print()

if results.p_value < 0.001:
    print("結論: p < 0.001 であり、高血圧の有無により")
    print("      心血管イベント発生率に統計的有意差が認められた。")
elif results.p_value < 0.05:
    print("結論: p < 0.05 であり、統計的有意差あり。")
else:
    print("結論: p >= 0.05 であり、統計的有意差なし。")

In [0]:
# 患者数上位5パターンを抽出
top_patterns = cohort['comorbidity_pattern'].value_counts().head(5).index.tolist()

# カラーパレット
colors = {
    '併存疾患なし': '#2ecc71',        # 緑
    '高血圧のみ': '#3498db',          # 青
    '脂質異常症のみ': '#f39c12',      # オレンジ
    '高血圧+脂質異常症': '#e74c3c',   # 赤
    '高血圧+慢性腎臓病': '#9b59b6',   # 紫
    '慢性腎臓病のみ': '#1abc9c',      # ティール
    '高血圧+脂質異常症+慢性腎臓病': '#c0392b'  # ダークレッド
}

fig, ax = plt.subplots(figsize=(12, 7))

for pattern in top_patterns:
    mask = cohort['comorbidity_pattern'] == pattern
    n = mask.sum()
    events = cohort.loc[mask, 'event'].sum()
    
    kmf.fit(
        cohort.loc[mask, 'follow_up_months'],
        cohort.loc[mask, 'event'],
        label=f'{pattern} (n={n:,}, events={events})'
    )
    color = colors.get(pattern, '#7f8c8d')
    kmf.plot_survival_function(ax=ax, ci_show=False, color=color, linewidth=2)

ax.set_xlabel('Follow-up (months)')
ax.set_ylabel('Event-free Survival Rate')
ax.set_title('Cardiovascular Event-free Survival by Comorbidity Pattern\n(Kaplan-Meier Curve)')
ax.legend(loc='lower left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.set_ylim(0.88, 1.0)

plt.tight_layout()
display(fig)
plt.close()

In [0]:
# 年齢層別のサブグループ解析
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

age_groups = [
    ('Under 70', cohort['age'] < 70),
    ('70 and over', cohort['age'] >= 70)
]

for idx, (title, age_mask) in enumerate(age_groups):
    ax = axes[idx]
    subset = cohort[age_mask]
    
    # 高血圧なし
    mask_no_ht = ~subset['has_hypertension']
    kmf.fit(
        subset.loc[mask_no_ht, 'follow_up_months'],
        subset.loc[mask_no_ht, 'event'],
        label=f"HT(-) (n={mask_no_ht.sum():,})"
    )
    kmf.plot_survival_function(ax=ax, ci_show=True, color='blue')
    
    # 高血圧あり
    mask_ht = subset['has_hypertension']
    kmf.fit(
        subset.loc[mask_ht, 'follow_up_months'],
        subset.loc[mask_ht, 'event'],
        label=f"HT(+) (n={mask_ht.sum():,})"
    )
    kmf.plot_survival_function(ax=ax, ci_show=True, color='red')
    
    # ログランク検定
    lr = logrank_test(
        subset.loc[mask_no_ht, 'follow_up_months'],
        subset.loc[mask_ht, 'follow_up_months'],
        subset.loc[mask_no_ht, 'event'],
        subset.loc[mask_ht, 'event']
    )
    
    ax.set_xlabel('Follow-up (months)')
    ax.set_ylabel('Event-free Survival Rate')
    ax.set_title(f'{title}\n(Log-rank p = {lr.p_value:.2e})')
    ax.legend(loc='lower left')
    ax.grid(True, alpha=0.3)
    ax.set_ylim(0.88, 1.0)

plt.suptitle('Subgroup Analysis by Age Group', fontsize=14, y=1.02)
plt.tight_layout()
display(fig)
plt.close()

In [0]:
# 結果のサマリー
print("=" * 70)
print("結果サマリー")
print("=" * 70)

# イベント率の計算
event_rate_no_ht = cohort.loc[~cohort['has_hypertension'], 'event'].mean() * 100
event_rate_ht = cohort.loc[cohort['has_hypertension'], 'event'].mean() * 100
rate_ratio = event_rate_ht / event_rate_no_ht

print(f"""
【対象コホート】
  対象者数: {len(cohort):,}人
  観察期間中央値: {cohort['follow_up_months'].median():.0f}ヶ月
  心血管イベント発生: {cohort['event'].sum():,}人 ({cohort['event'].mean()*100:.2f}%)

【患者背景】
  平均年齢: {cohort['age'].mean():.1f}歳
  男性割合: {(cohort['sex'] == '男').mean()*100:.1f}%
  高血圧併存率: {cohort['has_hypertension'].mean()*100:.1f}%
  慢性腎臓病併存率: {cohort['has_ckd'].mean()*100:.1f}%

【主要結果】
  心血管イベント発生率:
    高血圧なし群: {event_rate_no_ht:.2f}%
    高血圧あり群: {event_rate_ht:.2f}%
    発生率比: {rate_ratio:.2f}倍
  
  ログランク検定: p = {results.p_value:.2e}

【結論】
  高血圧を併存する糖尿病患者は、非併存患者と比較して
  心血管イベントの発生リスクが{rate_ratio:.1f}倍高く、
  統計的に有意な差が認められた (p < 0.001)。
  
  特に高血圧と慢性腎臓病を併存する患者群では
  イベント発生率が最も高く、積極的な予防介入が求められる。
""")